In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV

file_path = r'C:\python\Copper_Set.xlsx'
copper_data = pd.read_excel(file_path)
copper_data

,id,item_date,quantity tons,customer,country,status,item type,application,thickness,width,material_ref,product_ref,delivery date,selling_price
0,EC06F063-9DF0-440C-8764-0B0C05A4F6AE,20210401.0,54.151139,30156308.0,28.0,Won,W,10.0,2.00,1500.0,DEQ1 S460MC,1670798778,20210701.0,854.00
1,4E5F4B3D-DDDF-499D-AFDE-A3227EC49425,20210401.0,768.024839,30202938.0,25.0,Won,W,41.0,0.80,1210.0,0000000000000000000000000000000000104991,1668701718,20210401.0,1047.00
2,E140FF1B-2407-4C02-A0DD-780A093B1158,20210401.0,386.127949,30153963.0,30.0,Won,WI,28.0,0.38,952.0,S0380700,628377,20210101.0,644.33
3,F8D507A0-9C62-4EFE-831E-33E1DA53BB50,20210401.0,202.411065,30349574.0,32.0,Won,S,59.0,2.30,1317.0,DX51D+ZM310MAO 2.3X1317,1668701718,20210101.0,768.00
4,4E1C4E78-152B-430A-8094-ADD889C9D0AD,20210401.0,785.526262,30211560.0,28.0,Won,W,10.0,4.00,2000.0,2_S275JR+AR-CL1,640665,20210301.0,577.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181668,DE633116-D1DF-4846-982E-55EFC3658A76,20200702.0,102.482422,30200854.0,25.0,Won,W,41.0,0.96,1220.0,0000000000000000000000000000000001000777,164141591,20200701.0,591.00
181669,A48374B1-E6DB-45F2-889A-1F9C27C099EB,20200702.0,208.086469,30200854.0,25.0,Won,W,41.0,0.95,1500.0,0000000000000000000000000000000001000227,164141591,20200701.0,589.00
181670,91643238-5C7B-4237-9A5F-63AE3D35F320,20200702.0,4.235594,30200854.0,25.0,Won,W,41.0,0.71,1250.0,0000000000000000000000000000000001004216,164141591,20200701.0,619.00
181671,7AFFD323-01D9-4E15-B80D-7D1B03498FC8,20200702.0,-2000,30200854.0,25.0,Won,W,41.0,0.85,1250.0,0000000000000000000000000000000001001149,164141591,20200701.0,601.00


In [37]:
copper_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181673 entries, 0 to 181672
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             181671 non-null  object 
 1   item_date      181672 non-null  float64
 2   quantity tons  181673 non-null  object 
 3   customer       181672 non-null  float64
 4   country        181645 non-null  float64
 5   status         181671 non-null  object 
 6   item type      181673 non-null  object 
 7   application    181649 non-null  float64
 8   thickness      181672 non-null  float64
 9   width          181673 non-null  float64
 10  material_ref   103754 non-null  object 
 11  product_ref    181673 non-null  int64  
 12  delivery date  181672 non-null  float64
 13  selling_price  181672 non-null  float64
dtypes: float64(8), int64(1), object(5)
memory usage: 19.4+ MB


In [38]:
copper_data['material_ref'] = copper_data['material_ref'].apply(lambda x: np.nan if str(x).startswith('00000') else x)

In [39]:
#analyzing_null_value:
missing_values = copper_data.isnull().sum()
print("Total missing values by column:")
print(missing_values)

Total missing values by column:
id                    2
item_date             1
quantity tons         0
customer              1
country              28
status                2
item type             0
application          24
thickness             1
width                 0
material_ref     100645
product_ref           0
delivery date         1
selling_price         1
dtype: int64


In [40]:
#changing data types
copper_data["item_date"] = pd.to_datetime(copper_data["item_date"], format="%Y%m%d", errors="coerce")
copper_data["delivery date"] = pd.to_datetime(copper_data["delivery date"], format="%Y%m%d", errors="coerce")
copper_data["quantity tons"] = pd.to_numeric(copper_data["quantity tons"], errors="coerce")
copper_data["customer"] = pd.to_numeric(copper_data["customer"], errors="coerce")
copper_data["country"] = pd.to_numeric(copper_data["country"], errors="coerce")
copper_data["application"] = pd.to_numeric(copper_data["application"], errors="coerce")
copper_data['product_ref'] = pd.to_numeric(copper_data["product_ref"], errors="coerce")
copper_data.head(2)

,id,item_date,quantity tons,customer,country,status,item type,application,thickness,width,material_ref,product_ref,delivery date,selling_price
0,EC06F063-9DF0-440C-8764-0B0C05A4F6AE,2021-04-01,54.151139,30156308.0,28.0,Won,W,10.0,2.0,1500.0,DEQ1 S460MC,1670798778,2021-07-01,854.0
1,4E5F4B3D-DDDF-499D-AFDE-A3227EC49425,2021-04-01,768.024839,30202938.0,25.0,Won,W,41.0,0.8,1210.0,NaN,1668701718,2021-04-01,1047.0


In [41]:
copper_data.dtypes

id                       object
item_date        datetime64[ns]
quantity tons           float64
customer                float64
country                 float64
status                   object
item type                object
application             float64
thickness               float64
width                   float64
material_ref             object
product_ref               int64
delivery date    datetime64[ns]
selling_price           float64
dtype: object

In [42]:
#analyzing_null_value:
missing_values = copper_data.isnull().sum()
print("Total missing values by column:")
print(missing_values)

Total missing values by column:
id                    2
item_date             3
quantity tons         1
customer              1
country              28
status                2
item type             0
application          24
thickness             1
width                 0
material_ref     100645
product_ref           0
delivery date         3
selling_price         1
dtype: int64


In [43]:
copper_data.drop(columns=["material_ref"], inplace=True)

In [44]:
# Continuous variables
copper_data['quantity tons'] = copper_data['quantity tons'].fillna(copper_data['quantity tons'].mean())
copper_data['thickness'] = copper_data['thickness'].fillna(copper_data['thickness'].median())
copper_data['selling_price'] = copper_data['selling_price'].fillna(copper_data['selling_price'].mean())
# Categorical variables
copper_data['country'] = copper_data['country'].fillna(copper_data['country'].mode()[0])
copper_data['customer'] = copper_data['customer'].fillna(copper_data['customer'].mode()[0])
copper_data['application'] = copper_data['application'].fillna(copper_data['application'].mode()[0])
copper_data['status'] = copper_data['status'].fillna(copper_data['status'].mode()[0])

In [45]:
#analyzing_null_value:
missing_values = copper_data.isnull().sum()
print("Total missing values by column:")
print(missing_values)

Total missing values by column:
id               2
item_date        3
quantity tons    0
customer         0
country          0
status           0
item type        0
application      0
thickness        0
width            0
product_ref      0
delivery date    3
selling_price    0
dtype: int64


In [46]:
copper_data.dropna(inplace=True)

In [47]:
#analyzing_null_value:
missing_values = copper_data.isnull().sum()
print("Total missing values by column:")
print(missing_values)

Total missing values by column:
id               0
item_date        0
quantity tons    0
customer         0
country          0
status           0
item type        0
application      0
thickness        0
width            0
product_ref      0
delivery date    0
selling_price    0
dtype: int64


In [48]:
copper_data.head(5)

,id,item_date,quantity tons,customer,country,status,item type,application,thickness,width,product_ref,delivery date,selling_price
0,EC06F063-9DF0-440C-8764-0B0C05A4F6AE,2021-04-01,54.151139,30156308.0,28.0,Won,W,10.0,2.00,1500.0,1670798778,2021-07-01,854.00
1,4E5F4B3D-DDDF-499D-AFDE-A3227EC49425,2021-04-01,768.024839,30202938.0,25.0,Won,W,41.0,0.80,1210.0,1668701718,2021-04-01,1047.00
2,E140FF1B-2407-4C02-A0DD-780A093B1158,2021-04-01,386.127949,30153963.0,30.0,Won,WI,28.0,0.38,952.0,628377,2021-01-01,644.33
3,F8D507A0-9C62-4EFE-831E-33E1DA53BB50,2021-04-01,202.411065,30349574.0,32.0,Won,S,59.0,2.30,1317.0,1668701718,2021-01-01,768.00
4,4E1C4E78-152B-430A-8094-ADD889C9D0AD,2021-04-01,785.526262,30211560.0,28.0,Won,W,10.0,4.00,2000.0,640665,2021-03-01,577.00


In [49]:
copper_data.shape

(181665, 13)

In [53]:
#Treat Outliers using IQR
numeric_columns = ['quantity tons', 'thickness', 'width', 'product_ref', 'selling_price']
Q1 = copper_data[numeric_columns].quantile(0.25)
Q3 = copper_data[numeric_columns].quantile(0.75)
IQR = Q3 - Q1
threshold = 1.5
outliers = (copper_data[numeric_columns] < (Q1 - threshold * IQR)) | (copper_data[numeric_columns] > (Q3 + threshold * IQR))
copper_data[outliers] = None
print("Number of outliers detected and treated:")
print(outliers.sum())
for i in numeric_columns:
    plot(copper_data,i)

Number of outliers detected and treated:
quantity tons      0
thickness          0
width            738
product_ref        0
selling_price    210
dtype: int64


NameError: name 'plot' is not defined

In [51]:
print("Before transformation:")
copper_data.head()

Before transformation:


,id,item_date,quantity tons,customer,country,status,item type,application,thickness,width,product_ref,delivery date,selling_price
0,EC06F063-9DF0-440C-8764-0B0C05A4F6AE,2021-04-01,54.151139,30156308.0,28.0,Won,W,10.0,2.00,1500.0,1670798778,2021-07-01,854.00
1,4E5F4B3D-DDDF-499D-AFDE-A3227EC49425,2021-04-01,NaN,30202938.0,25.0,Won,W,41.0,0.80,1210.0,1668701718,2021-04-01,1047.00
2,E140FF1B-2407-4C02-A0DD-780A093B1158,2021-04-01,NaN,30153963.0,30.0,Won,WI,28.0,0.38,952.0,628377,2021-01-01,644.33
3,F8D507A0-9C62-4EFE-831E-33E1DA53BB50,2021-04-01,NaN,30349574.0,32.0,Won,S,59.0,2.30,1317.0,1668701718,2021-01-01,768.00
4,4E1C4E78-152B-430A-8094-ADD889C9D0AD,2021-04-01,NaN,30211560.0,28.0,Won,W,10.0,4.00,NaN,640665,2021-03-01,577.00


In [52]:
# Identify and handle skewness
skewed_features = copper_data[numeric_columns].apply(lambda x: abs(x.skew()))
skewed_features = skewed_features[skewed_features > 0.5]  # Adjust threshold as needed
for feature in skewed_features.index:
    copper_data[feature] = np.log1p(copper_data[feature] + 1e-6)

print("\nAfter transformation:")
copper_data.head()


After transformation:


C:\Users\Asus\AppData\Roaming\Python\Python312\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


,id,item_date,quantity tons,customer,country,status,item type,application,thickness,width,product_ref,delivery date,selling_price
0,EC06F063-9DF0-440C-8764-0B0C05A4F6AE,2021-04-01,4.010077,30156308.0,28.0,Won,W,10.0,1.098613,1500.0,21.236568,2021-07-01,854.00
1,4E5F4B3D-DDDF-499D-AFDE-A3227EC49425,2021-04-01,NaN,30202938.0,25.0,Won,W,41.0,0.587787,1210.0,21.235312,2021-04-01,1047.00
2,E140FF1B-2407-4C02-A0DD-780A093B1158,2021-04-01,NaN,30153963.0,30.0,Won,WI,28.0,0.322084,952.0,13.350897,2021-01-01,644.33
3,F8D507A0-9C62-4EFE-831E-33E1DA53BB50,2021-04-01,NaN,30349574.0,32.0,Won,S,59.0,1.193923,1317.0,21.235312,2021-01-01,768.00
4,4E1C4E78-152B-430A-8094-ADD889C9D0AD,2021-04-01,NaN,30211560.0,28.0,Won,W,10.0,1.609438,NaN,13.370264,2021-03-01,577.00
